In [1]:
from sdv import Metadata
import os

print(os.getcwd())
from test_20_tables import fetch_data_from_sqlite_filter

metadata, tables = fetch_data_from_sqlite_filter(["BookLoan", "Book", "Library", "Student", "Enrollment", "Submission"],"1k_data_sqlite.db")
metadata = Metadata(metadata)
metadata

C:\Users\Elvin\Desktop\GitHub\SDV\tutorials\relational_data


Metadata
  root_path: .
  tables: ['BookLoan', 'Book', 'Library', 'Student', 'Enrollment', 'Submission']
  relationships:
    BookLoan.book_id -> Book.book_id
    BookLoan.student_id -> Student.student_id
    Book.library_id -> Library.library_id
    Enrollment.student_id -> Student.student_id
    Submission.student_id -> Student.student_id

In [2]:
metadata.to_json("1.json")

In [3]:
for key,table in tables.items():
    print(table.head(1))

   loan_id  book_id  student_id   loan_date return_date
0        1        1         735  2024-09-30  2025-07-08
   book_id               title           author               isbn  library_id
0        1  Cause source blue.  Pamela Mccarthy  978-0-385-99866-6           1
   library_id                     name                                location
0           1  Wright-Thompson Library  9510 Medina Valley\nLisatown, RI 01200
   student_id             name                     email date_of_birth major
0           1  Clarence Spence  robertgrimes@example.com    1996-11-25    70
   enrollment_id  student_id  course_id     semester grade
0              1         934        143  Spring 2019  None
   submission_id  assignment_id  student_id      submission_date grade
0              1             67         875  2022-11-24 21:49:55     C


In [4]:
from sdv.relational import HMA1
import warnings
warnings.filterwarnings('ignore')

model = HMA1(metadata)    

In [5]:
model.fit(tables)

Fitting HMA1: 0it [00:00, ?it/s]

Modeling Library
Modeling Book
Modeling BookLoan


Book:BookLoan:   0%|          | 0/1 [00:00<?, ?it/s]

BookLoan Unique:   0%|          | 0/1000 [00:00<?, ?it/s]

Library:Book:   0%|          | 0/1 [00:00<?, ?it/s]

...

Student:Submission:   0%|          | 0/1 [00:00<?, ?it/s]

Submission Unique:   0%|          | 0/659 [00:00<?, ?it/s]

Modeling Enrollment


Student:Enrollment:   0%|          | 0/1 [00:00<?, ?it/s]

Enrollment Unique:   0%|          | 0/619 [00:00<?, ?it/s]

In [6]:
model.save('my_model_1k.pkl')

In [7]:
new_data = model.sample(num_rows=1000)
new_data

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Library Childing:   0%|          | 0/1 [00:00<?, ?it/s]

Book:   0%|          | 0/1000 [00:00<?, ?it/s]

Book Childing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Enrollment:   0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

BookLoan:   0%|          | 0/2 [00:00<?, ?it/s]

BookLoan student_id Likelihoods:   0%|          | 0/1000 [00:00<?, ?it/s]

{'Library':      library_id                                   name  \
 0             0                      Smith Inc Library   
 1             1                 Gardner-Garcia Library   
 2             2                 Davis-Cantrell Library   
 3             3                   Perez-Hodges Library   
 4             4      Meyer, Cooke and Santiago Library   
 ..          ...                                    ...   
 995         995                    Dickson PLC Library   
 996         996                     Sparks PLC Library   
 997         997                  Palmer-Larsen Library   
 998         998                    Luna-Parker Library   
 999         999  Bradford, Turner and Calderon Library   
 
                                               location  
 0               9510 Medina Valley\nLisatown, RI 01200  
 1          5454 Stephanie Cliff\nNorth James, GA 29360  
 2               679 Bush Villages\nParksland, MA 63600  
 3          808 Anna Underpass\nLake Jeremiah, 

In [9]:
synthetic_data = new_data
real_data = tables

from sdv.evaluation import evaluate
evaluate(synthetic_data, real_data,metadata=metadata, aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSComplement,Inverted Kolmogorov-Smirnov D statistic,0.916713,0.916713,0.0,1.0,MAXIMIZE,None
1,CSTest,Chi-Squared,0.970035,0.970035,0.0,1.0,MAXIMIZE,None


In [10]:
evaluate(synthetic_data, real_data,metadata=metadata)

0.9433740938332372